**LLM 的参数裁剪（Pruning）**

剪枝的分类如下。

- 非结构化剪枝（Unstructured Pruning）
	- 按权重绝对值大小直接将接近 0 的参数置零。
	- 硬件利用率不佳，稀疏矩阵乘法加速效果有限（除非稀疏率 > 90%）；
	- 常用策略
		- Magnitude pruning：按 |W| 排序，剪去最小的一部分；
		- Gradual pruning：训练过程中逐步提高稀疏度，避免性能骤降；
		- Movement pruning（Google 提出）：考虑权重在训练过程中的更新方向，保留更可能有用的参数；

- 结构化剪枝（Structured Pruning）
	- 按一定结构单位整体移除，如整个神经元、卷积通道、注意力头、MLP 中的隐藏维度块；
	- 对硬件友好，可直接减少计算 FLOPs；
	- 常用策略
		- 剪掉低重要度的注意力头（如部分 head 对输出影响极小）；
		- 剪掉激活较少的 FFN 中间维度块；
		- 剪掉整个 Transformer 层（layer dropping）；

- 动态剪枝（Dynamic Pruning / Conditional Computation）
	- 方式：推理时动态决定哪些参数或模块参与计算；
	- 代表方法
		- SparseMoE（Mixture of Experts）：只激活少量专家网络；这里可以看到其实 MoE 本身就可以算作一种剪枝；
		- Dynamic Sparsity：输入相关的稀疏模式（token-level gating）；
		- Attention head routing：对不同 token 动态选择不同的 head；

剪枝的常用技术流程包括以下。

- 重要性评估
	- 权重大小（magnitude）；
	- 梯度敏感性（Taylor expansion）；
	- Fisher 信息量；
	- L1 / L2 范数、激活值统计；
- 执行剪枝
	- 一次性剪（one-shot）；
	- 逐步剪（iterative）；
- 微调恢复精度
	- 全参微调（Full fine-tune）；
	- LoRA/Adapter 微调；
- 稀疏加速部署
	- 使用支持稀疏计算的库；
	- 如 NVIDIA cuSPARSE、DeepSpeed Sparse Attention、Torch Sparse）；

在统计学里，Fisher 信息量衡量的是观测数据对参数 θ 提供了多少信息。

- 方差表示，$f(X;\theta)$ 是观测 X 关于参数 θ 的概率密度函数，分母隐含在期望中
$$\mathcal{I}(\theta) = E\left[\left(\frac{\partial}{\partial\theta} \log f(X; \theta)\right)^2\right]$$

- 负二阶导数形式
$$\mathcal{I}(\theta) = -E\left[\frac{\partial^2}{\partial\theta^2} \log f(X; \theta)\right]$$

- 多参数情形，则扩展为Fisher 信息矩阵（FIM），其第 (i,j) 元素为
$$[\mathcal{I}(\theta)]_{i,j} = E\left[\frac{\partial}{\partial\theta_i} \log f(X;\theta)\cdot\frac{\partial}{\partial\theta_j} \log f(X;\theta)\right]$$


在 LLM 剪枝中，Fisher 信息通常被用于衡量每个参数（或结构单元）对模型损失变化的敏感度，从而决定是否可以安全地删除它。
- 近似 Hessian 的替代：直接求 Hessian 矩阵非常昂贵，Fisher 信息矩阵常被用来近似 Hessian，这在某些剪枝算法中用于估算 loss 的二阶变化；
- 使用 FIM trace 来衡量层的敏感度：比如在极端剪枝方法中，通过计算每层 Fisher 信息矩阵的迹（trace）来定量评估某层对 loss 的影响程度，以指导剪枝优先级；